In [ ]:
import matplotlib as mpl
mpl.use('Agg')
mpl.rcParams['animation.embed_limit'] = 300  # 例: 50MBに増やす
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
from matplotlib import animation
import math
from IPython.display import HTML
import os
from matplotlib.animation import PillowWriter

%matplotlib inline

無次元系への変換

In [ ]:
# =====================================================
# 1. 物理パラメータ（元コードと同じ形）
# =====================================================

# --- 進行波・電極 ---
Lx     = 0.08          # [m] x方向長さ
waves  = 1.0           # 波数（何波入っているか）
wave_v = 0.5           # [m/s] 波の位相速度
lambda_ = Lx / waves   # [m] 波長
f      = wave_v / lambda_           # [Hz]
k_wave = 2.0 * math.pi / lambda_    # [1/m]
omega  = 2.0 * math.pi * f          # [rad/s]

V0     = 1000.0        # [V] 電極電位振幅
phi0   = 0.0           # [rad] 初期位相
ground_z = 0.0         # [m] 地面高さ

# --- 定数 ---
g      = 1.625         # [m/s^2] 月面重力
eps_0  = 8.85e-12      # [F/m] 真空誘電率

# --- 粒子（代表値）---
R_phys = 3e-5          # [m] 粒子半径（例: 30 µm）
m_phys = 4e-10         # [kg] 粒子質量（例: R=30µmレゴリス相当）
q_phys = 6.5e-14         # [C] 粒子電荷

# --- 摩擦（物理的な係数）---
mu_s_phys = 10        # 静止摩擦係数（例）
mu_k_phys = 10        # 動摩擦係数（例）

# =====================================================
# 2. 無次元スケールと γ, β, Ω の計算
# =====================================================

# 代表電場スケール E0 ≈ k V0
E0 = k_wave * V0   # [V/m]

# 時間スケール T0: クーロン力と慣性のバランスから
#   T0^2 = m / (q k^2 V0)
T0 = math.sqrt(m_phys / (q_phys * (k_wave**2) * V0))

# 無次元パラメータ
gamma_phys = (m_phys * g) / (q_phys * E0)
beta_phys  = (q_phys * k_wave) / (16.0 * math.pi * eps_0 * V0)
Omega_phys = omega * T0



print("=== Physical parameters ===")
print(f"Lx      = {Lx:.3e} m")
print(f"lambda  = {lambda_:.3e} m")
print(f"k_wave  = {k_wave:.3e} 1/m")
print(f"omega   = {omega:.3e} rad/s")
print(f"V0      = {V0:.3e} V")
print(f"R       = {R_phys:.3e} m")
print(f"m       = {m_phys:.3e} kg")
print(f"q       = {q_phys:.3e} C")
print()
print("=== Derived scales ===")
print(f"E0      = {E0:.3e} V/m")
print(f"T0      = {T0:.3e} s  (time scale)")
print()
print("=== Dimensionless parameters from these physical values ===")
print(f"gamma   = {gamma_phys:.3e}   (gravity / electric)")
print(f"beta    = {beta_phys:.3e}   (image / electric)")
print(f"Omega   = {Omega_phys:.3e}   (omega * T0)")

# =====================================================
# 2.5  位置・半径の無次元化（確認用）
# =====================================================

# 長さスケール L0 = 1/k
L0 = 1.0 / k_wave

def x_hat_from_phys(x_phys):
    """物理位置 x[m] を無次元座標 x_hat に変換"""
    return x_phys / L0   # = k_wave * x_phys

def z_hat_from_phys(z_phys):
    return z_phys / L0

def R_hat_from_phys(R_phys_val):
    return R_phys_val / L0

# 例：初期位置 x0 = 0.02 m, z0 = 0.005 m のときの無次元量
x0_phys = 0.02   # [m]
z0_phys = 0.005  # [m]

x0_hat = x_hat_from_phys(x0_phys)
z0_hat = z_hat_from_phys(z0_phys)
R_hat  = R_hat_from_phys(R_phys)
ground_z_hat = z_hat_from_phys(ground_z)

print()
print("=== Example: dimensionless position / radius ===")
print(f"x0_phys = {x0_phys:.5e} m -> x_hat = {x0_hat:.5e}")
print(f"z0_phys = {z0_phys:.5e} m -> z_hat = {z0_hat:.5e}")
print(f"R_phys  = {R_phys:.5e} m -> R_hat = {R_hat:.5e}")
print(f"ground_z = {ground_z:.5e} m -> ground_z_hat = {ground_z_hat:.5e}")


定数

In [ ]:
# =====================================================
# 3. 無次元シミュレーション用パラメータ
# =====================================================

# # # ここから下で、設定
#定数
R_hat = 5e-3    #粒子半径の無次元量
c_hat = R_hat     #地面に接している時、鏡像力の最大値の距離
gamma = 0.7     #重力/最大クーロン力
beta  = 1e-5    #鏡像力/最大クーロン力
Omega = 1.45     #角周波数×時間スケール 波速度/粒子が最大クーロン力から受ける速度
mu_k_hat = 0.2    #動摩擦係数の無次元量
mu_s_hat = 0.5


print()
print("=== Dimensionless parameters actually used in simulation ===")
print(f"gamma(sim) = {gamma:.3e}")
print(f"beta(sim)  = {beta:.3e}")
print(f"Omega(sim) = {Omega:.3e}")
print(f"R_hat(sim) = {R_hat:.3e}")
print(f"c_hat(sim) = {c_hat:.3e}")

粒子クラス

In [ ]:
class Particle:
    """
    無次元シミュレーション用の粒子。
    x, z, vx, vz, t などはすべて「無次元量」として扱う。
    （物理単位への変換はここでは一切行わない）
    """
    def __init__(self, x0, z0, vx0=0.0, vz0=0.0, R=1.0, label="", color=None):
        # --- 現在状態（すべて無次元量） ---
        self.x  = float(x0)
        self.z  = float(z0)
        self.vx = float(vx0)
        self.vz = float(vz0)
        self.R  = float(R)   # 粒子半径（無次元）

        self.label = label
        self.color = color

        # --- 接触・イベント関係 ---
        self.in_contact = False          # 地面と接触しているか
        self.bounce_count = 0            # 跳ねた回数
        self.collision_times = []        # 衝突した無次元時刻リスト

        # --- 履歴（無次元量）---
        self.hist = {
            "t":  [],
            "x":  [],
            "z":  [],
            "vx": [],
            "vz": [],

            # x方向
            "ax_tot":   [],   # 合力 a_x
            "ax_E":     [],   # 電場（クーロン）による a_x
            "ax_fric":  [],   # 摩擦による a_x（静止 or 動摩擦）
            # （必要なら）他の x 方向成分が出てきたときに拡張

            # z方向
            "az_tot":   [],   # 合力 a_z
            "az_E":     [],   # 電場（クーロン）による a_z
            "az_img":   [],   # 鏡像力による a_z
            "az_g":     [],   # 重力による a_z

            # 接触情報
            "N_hat":        [],   # 法線反力（無次元）
            "in_contact":   [],   # 地面と接触しているか（True/False を 0/1 でも可）

        }

    def save(self,
             t_hat,
             ax_tot=0.0, az_tot=0.0,
             ax_E=0.0,   az_E=0.0,
             az_img=0.0, az_g=0.0,
             ax_fric=0.0,
             N_hat=0.0,
             in_contact=False):
        """
        無次元時間 t における現在状態を履歴に保存
        """
        self.hist["t"].append(float(t_hat))
        self.hist["x"].append(self.x)
        self.hist["z"].append(self.z)
        self.hist["vx"].append(self.vx)
        self.hist["vz"].append(self.vz)

        # 加速度・接触情報
        self.hist["ax_tot"].append(ax_tot)
        self.hist["az_tot"].append(az_tot)
        self.hist["ax_E"].append(ax_E)
        self.hist["az_E"].append(az_E)
        self.hist["az_img"].append(az_img)
        self.hist["az_g"].append(az_g)
        self.hist["ax_fric"].append(ax_fric)
        self.hist["N_hat"].append(N_hat)
        self.hist["in_contact"].append(1 if in_contact else 0)

# -------------------------------------------------
    # 物理量から直接 Particle を作るクラスメソッド
    # -------------------------------------------------
    @classmethod
    def from_physical(cls,
                      x0_phys, z0_phys,
                      vx0=0.0, vz0=0.0,
                      R_phys_val=R_phys,
                      label="",
                      color=None):
        """
        物理座標 (x0_phys [m], z0_phys [m], R_phys [m]) から
        無次元粒子を生成するヘルパー。
        速度 vx0, vz0 は「すでに無次元化された値」として与える前提。
        （もし速度も物理単位から無次元化したくなったら、そのとき専用関数を追加）
        """
        x_hat0 = x_hat_from_phys(x0_phys)
        z_hat0 = z_hat_from_phys(z0_phys)
        R_hat  = R_hat_from_phys(R_phys_val)

        return cls(
            x0=x_hat0,
            z0=z_hat0,
            vx0=vx0,
            vz0=vz0,
            R=R_hat,
            label=label,
            color=color,
        )

関数

In [ ]:
# =====================================================
# 5. 無次元運動方程式：各力の項
# =====================================================
### SI単位の物理量使う時
# EPS_CONTACT_phys = max(1e-10, 1e-5 * R_phys)  # 物理単位での接触判定用しきい値
# EPS_CONTACT = EPS_CONTACT_phys / L0  # 無次元化したもの
# VEL_EPS_phys = 1e-9 # [m/s] 物理単位での速度ゼロ判定用しきい値
# VEL_EPS = VEL_EPS_phys / (L0 / T0)  # 無次元化したもの

### 無次元量使う時
contact_rel = 1e-4
EPS_CONTACT = contact_rel * R_hat

VEL_EPS = 1e-6

def electric_accel(p: Particle, t: float):
    """
    クーロン力（進行波電場）による無次元加速度項 (a_x, a_z) を返す。
    t は無次元時間。
    """
    phase = p.x - Omega * t + phi0
    exp_factor = math.exp(-p.z)

    ax = -exp_factor * math.cos(phase)
    az =  exp_factor * math.sin(phase)
    return ax, az

def image_accel(p: Particle):
    """
    鏡像力による無次元加速度項 (a_x, a_z) を返す。
    x方向成分は 0、z方向のみ。
    """
    # z が 0 に近づきすぎると 1/z^2 が発散するので、最小値を入れておく
    z_eff = (p.z - p.R) + c_hat - ground_z_hat

    s_min = 0.1 * p.R  # とりあえず 0.1R など
    if z_eff < s_min:
        z_eff = s_min

    az = -beta / (z_eff**2)
    return 0.0, az

def gravity_accel():
    """
    重力による無次元加速度項 (a_x, a_z) を返す。
    x 方向は 0、z 方向のみ一定。
    """
    az = -gamma
    return 0.0, az


def contact_and_friction_accel(p: Particle, t: float):
    """
    接触判定＋静止摩擦／動摩擦込みの加速度を計算する。
    戻り値:
        ax_total, az_total, in_contact, N_hat
    """
    # まず「電場＋鏡像力＋重力」の項を計算
    ax_e, az_e = electric_accel(p, t)
    _,   az_img = image_accel(p)
    _,   az_g   = gravity_accel()

    az_free = az_e + az_img + az_g

    # 接触判定（粒子下端が地面を下回っていれば接触とみなす）
    in_contact = ((p.z - p.R) <= (ground_z_hat + EPS_CONTACT))

    N_hat = 0.0
    az_total = az_free

    # --- 法線反力の計算 ---
    if in_contact and az_free < 0.0:
        # 下向きに加速しようとしているので、床が押し返す
        # z方向加速度を 0 にするように反力を入れる
        N_hat = -az_free
        az_total = 0.0
    else:
        # それ以外は反力なし（浮いている or 上向き加速）
        N_hat = 0.0
        az_total = az_free
        # 浮き上がるときは接触フラグを落とすことも考えられる
        if in_contact and az_free >= 0.0:
            in_contact = False

    # --- x方向：摩擦 ---
    ax_fric  = 0.0        # まず摩擦は0としておく
    ax_total = ax_e

    if in_contact:
        # 静止摩擦条件のチェック
        # 1. 速度がほぼゼロ
        # 2. クーロン力による加速度が静止摩擦限界以下
        if (abs(p.vx) < VEL_EPS) and (abs(ax_e) <= mu_s_hat * N_hat):
            # 静止摩擦：滑り出さないように完全に打ち消す
            ax_fric  = -ax_e      # 電場を打ち消す向き・大きさ
            ax_total = ax_e + ax_fric   # = 0
        else:
            # 動摩擦：速度方向と逆向きに mu_k * N_hat を加える
            if abs(p.vx) > VEL_EPS:
                sign = math.copysign(1.0, p.vx)
            else:
                # 速度がほぼ 0 なら、電場による加速方向に滑り出すと仮定
                sign = math.copysign(1.0, ax_e) if ax_e != 0.0 else 0.0

            ax_fric  = -mu_k_hat * N_hat * sign
            ax_total = ax_e + ax_fric
            # ax_total = 0.0  #摩擦力大きくする時

    return ax_total, az_total, in_contact, N_hat, ax_fric


def accel_total(p: Particle, t: float):
    """
    粒子 p の無次元加速度 (ax, az) を返す高水準関数。
    - クーロン力
    - 鏡像力
    - 重力
    - 接触・静止摩擦・動摩擦
    をすべて含めたもの。
    """
    ax, az, in_contact, N_hat, ax_fric = contact_and_friction_accel(p, t)

    # 接触状態を粒子にも反映しておく（イベント判定などに使える）
    p.in_contact = in_contact
    return ax, az


# =====================================================
# RK4 1ステップ分
# =====================================================

def rk4_step(p: Particle, t: float, dt: float):
    """
    粒子 p を、無次元時間 t から t+dt まで RK4 で1ステップ進める。
    p の状態 (x, z, vx, vz) を in-place で更新する。
    """

    # 現在状態を退避しておく
    x0, z0 = p.x, p.z
    vx0, vz0 = p.vx, p.vz
    in_contact0 = p.in_contact

    # accel_total を「任意の状態 (x,z,vx,vz,t) に対して」使うためのヘルパ
    def deriv(x, z, vx, vz, t_local):
        """
        状態 (x,z,vx,vz) における微分:
            dx/dt = vx
            dz/dt = vz
            dvx/dt, dvz/dt = accel_total(...)
        を返す。
        """
        # p を一時的にこの状態にして accel_total を呼ぶ
        p.x, p.z, p.vx, p.vz = x, z, vx, vz
        ax, az = accel_total(p, t_local)

        # 状態を元に戻す（p.in_contact も戻す）
        p.x, p.z, p.vx, p.vz = x0, z0, vx0, vz0
        p.in_contact = in_contact0

        return vx, vz, ax, az

    # k1
    k1_x, k1_z, k1_vx, k1_vz = deriv(x0, z0, vx0, vz0, t)

    # k2
    x2  = x0  + 0.5 * dt * k1_x
    z2  = z0  + 0.5 * dt * k1_z
    vx2 = vx0 + 0.5 * dt * k1_vx
    vz2 = vz0 + 0.5 * dt * k1_vz
    k2_x, k2_z, k2_vx, k2_vz = deriv(x2, z2, vx2, vz2, t + 0.5 * dt)

    # k3
    x3  = x0  + 0.5 * dt * k2_x
    z3  = z0  + 0.5 * dt * k2_z
    vx3 = vx0 + 0.5 * dt * k2_vx
    vz3 = vz0 + 0.5 * dt * k2_vz
    k3_x, k3_z, k3_vx, k3_vz = deriv(x3, z3, vx3, vz3, t + 0.5 * dt)

    # k4
    x4  = x0  + dt * k3_x
    z4  = z0  + dt * k3_z
    vx4 = vx0 + dt * k3_vx
    vz4 = vz0 + dt * k3_vz
    k4_x, k4_z, k4_vx, k4_vz = deriv(x4, z4, vx4, vz4, t + dt)

    # RK4 更新
    p.x  = x0  + (dt / 6.0) * (k1_x  + 2.0 * k2_x  + 2.0 * k3_x  + k4_x)
    p.z  = z0  + (dt / 6.0) * (k1_z  + 2.0 * k2_z  + 2.0 * k3_z  + k4_z)
    p.vx = vx0 + (dt / 6.0) * (k1_vx + 2.0 * k2_vx + 2.0 * k3_vx + k4_vx)
    p.vz = vz0 + (dt / 6.0) * (k1_vz + 2.0 * k2_vz + 2.0 * k3_vz + k4_vz)

    # 最後に、この新しい状態に対して in_contact を更新しておく
    now_in_contact = (p.z - p.R) <= (ground_z_hat + EPS_CONTACT)
    p.in_contact = now_in_contact



def t_end_hat_from_cycles(N_cycles, Omega):
    """
    N_cycles：何周期見るか
    Omega：無次元角周波数
    """
    T_hat = 2.0 * math.pi / Omega         # 無次元1周期
    t_end_hat = N_cycles * T_hat          # 無次元時間の合計
    return t_end_hat



def choose_dt_hat_for_Omega(Omega: float):
    """
    Ω に応じて、無次元時間ステップ dt_far/mid/near を決めるヘルパー。
    目安:
      - 遠方:   1周期あたり ~600ステップ
      - 中間:   ~6000ステップ
      - 近接:  ~60000ステップ
    """

    # 安全側の係数（必要なら後で変えてOK）
    fac_far  = 1e-2   # 0.01 / Ω
    fac_mid  = 1e-3   # 0.001 / Ω
    fac_near = 1e-4   # 0.00001 / Ω

    dt_far_hat  = fac_far  / Omega
    dt_mid_hat  = fac_mid  / Omega
    dt_near_hat = fac_near / Omega

    # 履歴保存間隔
    save_dt_hat = 5*dt_near_hat

    return dt_far_hat, dt_mid_hat, dt_near_hat, save_dt_hat


def simulate(
    p: Particle,
    t_end_hat: float,
    dt_far_hat: float,
    dt_mid_hat: float,
    dt_near_hat: float,
    save_dt_hat: float | None = None,
    ):
    """
    物理時間 [s] ベースの設定（t_end, dt_far, dt_mid, dt_near）を引数に取り、
    内部で無次元時間に変換してシミュレーションする。
    """

    t_hat  = 0.0            # 無次元時間

    # 初期接触状態
    in_contact = (p.z - p.R) <= (ground_z_hat + EPS_CONTACT)
    last_bounce_time = -1e9          # あり得ないくらい小さい値で初期化
    min_bounce_interval = 1        # これが「同一衝突とみなす時間幅」
    p.in_contact = in_contact
    prev_in_contact = in_contact

    # --- 初期状態の加速度を計算して保存 ---
    ax_E0, az_E0 = electric_accel(p, t_hat)
    _,   az_img0 = image_accel(p)
    _,   az_g0   = gravity_accel()
    ax_tot0, az_tot0, in_contact0, N_hat0, ax_fric0 = contact_and_friction_accel(p, t_hat)


    # 初期状態を保存
    p.save(
        t_hat,
        ax_tot=ax_tot0,
        az_tot=az_tot0,
        ax_E=ax_E0,
        az_E=az_E0,
        ax_fric=ax_fric0,
        az_img=az_img0,
        az_g=az_g0,
        N_hat=N_hat0,
        in_contact=in_contact0,
    )
    events = []
    time_since_save_hat = 0.0

    while t_hat < t_end_hat:
        # ---- 元コードと同じロジックで dt_hat を決める ----
        # ここは z, R, EPS_CONTACT を無次元 → 物理に戻してもいいし、
        # 無次元のまま「EPS_CONTACT も無次元」にして比較してもいい。
        if (p.z - p.R) > 50.0 * EPS_CONTACT:
            dt_hat = dt_far_hat
        elif (p.z - p.R) > 10.0 * EPS_CONTACT:
            dt_hat = dt_mid_hat
        else:
            dt_hat = dt_near_hat

        if t_hat + dt_hat > t_end_hat:
            dt_hat = t_end_hat - t_hat

        vz_before = p.vz

        # 1ステップ進める（無次元時間で）
        rk4_step(p, t_hat, dt_hat)
        t_hat  += dt_hat

        # 地面より下にめり込んだ場合の補正などは今の simulate と同じでOK
        if (p.z - p.R) < ground_z_hat:
            p.z = ground_z_hat + p.R
            p.vz = 0.0
            # p.vx = 0.0  #摩擦力大きくする時

        now_in_contact = (p.z - p.R) <= (ground_z_hat + EPS_CONTACT)
        p.in_contact = now_in_contact

        if (not prev_in_contact) and now_in_contact:
            if (vz_before < 0.0) and (t_hat - last_bounce_time > min_bounce_interval):
                p.bounce_count += 1
                p.collision_times.append(t_hat)  # 物理時間で記録してもいい
                events.append({"time": t_hat, "type": "bounce"})
                last_bounce_time = t_hat   # ここを必ず更新！

        if prev_in_contact and (not now_in_contact):
            events.append({"time": t_hat, "type": "lift-off"})

        prev_in_contact = now_in_contact

        # 履歴保存（ここでは無次元時間で保存している）
        time_since_save_hat += dt_hat
        if (save_dt_hat is None) or (time_since_save_hat >= save_dt_hat):
            # 現在状態で各加速度成分を計算
            ax_E, az_E = electric_accel(p, t_hat)
            _,   az_img = image_accel(p)
            _,   az_g   = gravity_accel()
            ax_tot, az_tot, in_contact_cf, N_hat, ax_fric = contact_and_friction_accel(p, t_hat)

            p.save(
                t_hat,
                ax_tot=ax_tot,
                az_tot=az_tot,
                ax_E=ax_E,
                az_E=az_E,
                ax_fric=ax_fric,
                az_img=az_img,
                az_g=az_g,
                N_hat=N_hat,
                in_contact=in_contact_cf,
            )
            
            time_since_save_hat = 0.0

    return p.hist, events


def compute_energy(hist_nd, Omega, R_hat, phi0=0.0):
    """
    無次元履歴 hist_nd から
      ψ, ψ', V(ψ), H = 1/2 ψ'^2 + A(1 - sinψ)
    を計算して返す。
    """
    t_hat = np.array(hist_nd["t"])
    x_hat = np.array(hist_nd["x"])
    vx_hat = np.array(hist_nd["vx"])

    # 位相差 ψ, 速度差 ψ' = v_x - Ω
    psi     = x_hat - Omega * t_hat + phi0
    psi_dot = vx_hat - Omega

    # 振幅 A ≃ e^{-R̂}（z≃R̂ に固定した近似）
    A = np.exp(-R_hat)

    # 有効ポテンシャル（定数シフト 1 は見た目のためだけで物理的には不要）
    # V = A * (1.0 - np.sin(psi))
    V = A * np.sin(psi)

    # 有効エネルギー
    H = 0.5 * psi_dot**2 + V

    return t_hat, psi, psi_dot, V, H


輸送効率用関数

In [ ]:
#周期境界の排除
def unwrap_periodic(x_arr, Lx=2*np.pi):
    """
    x が [0, Lx) の周期境界だと仮定して unwrap する簡易関数。
    すでに同様の関数がある場合は、そちらを使ってもよい。
    """
    x = np.array(x_arr, dtype=float)
    dx = np.diff(x)
    x_unwrapped = x.copy()
    offset = 0.0

    for j in range(1, len(x)):
        if abs(x[j] - x[j-1]) > Lx/2:
            # 境界をまたいだと判断
            if x[j] < x[j-1]:
                offset += Lx
            else:
                offset -= Lx
        x_unwrapped[j] += offset

    return x_unwrapped

#各粒子の平均速度の算出
# 各粒子の平均速度の算出（最初の数周期を捨てるオプション付き）
def compute_mean_velocity_from_hist(hist_nd, Lx=2*np.pi, discard_fraction: float = 0.0):
    """
    1 粒子の履歴 hist_nd から、長時間平均速度 v_mean を計算する。
    v_mean = (x(T) - x(t_use_start)) / (T - t_use_start)

    x は周期境界付きの位相座標を想定しているので、
    必要に応じて unwrap してから差分をとる。

    discard_fraction:
        0.0 なら全区間を使用
        0.3 なら、全時間のうち最初の 30% を捨てて残り 70% だけで平均速度をとる
    """
    t = np.array(hist_nd["t"], dtype=float)
    x = np.array(hist_nd["x"], dtype=float)

    if len(t) < 2:
        return 0.0

    # 周期境界をほどく
    x_unw = unwrap_periodic(x, Lx=Lx)

    # 使う時間区間の決定
    t0 = t[0]
    t1 = t[-1]
    dt_total = t1 - t0
    if dt_total <= 0:
        return 0.0

    if discard_fraction <= 0.0:
        # 何も捨てない
        use_mask = np.ones_like(t, dtype=bool)
    else:
        # 全体時間の discard_fraction ぶんを捨てる
        t_use_start = t0 + discard_fraction * dt_total
        use_mask = t >= t_use_start

        # 万一、点数が少なすぎる場合は全区間にフォールバック
        if np.sum(use_mask) < 2:
            use_mask = np.ones_like(t, dtype=bool)

    t_use = t[use_mask]
    x_use = x_unw[use_mask]

    dt_use = t_use[-1] - t_use[0]
    if dt_use <= 0:
        return 0.0

    v_mean = (x_use[-1] - x_use[0]) / dt_use
    return v_mean


#エネルギー的にトラップされているか
def is_synchronous_by_energy(hist_nd,
                             Omega_val: float,
                             R_hat_val: float,
                             phi0: float = 0.0,
                             margin: float = 0.02,
                             discard_fraction: float = 0.0) -> bool:
    """
    エネルギー H がセパラトリクス H = A の内側に
    （十分時間が経ったあとの区間で）とどまっているかどうかで
    「同期（トラップ）」判定する。

    条件:
        max_{late}(H) <= A * (1 + margin)

    discard_fraction:
        0.0 : 全時間で max(H) をとる（従来と同じ）
        0.3 : 全体時間のうち最初の 30% を捨てた残りで max(H) をとる
    """

    # 既存の compute_energy を使う
    t_hat, psi, psi_dot, V, H = compute_energy(
        hist_nd,
        Omega=Omega_val,
        R_hat=R_hat_val,
        phi0=phi0
    )

    A = np.exp(-R_hat_val)

    # --- 使う時間区間を決める（平均速度と同じロジック） ---
    if len(t_hat) < 2:
        H_use = H
    else:
        t0 = t_hat[0]
        t1 = t_hat[-1]
        dt_total = t1 - t0

        if dt_total <= 0 or discard_fraction <= 0.0:
            # そのまま全区間を使う
            H_use = H
        else:
            t_use_start = t0 + discard_fraction * dt_total
            use_mask = t_hat >= t_use_start

            # 点が少なすぎる場合はフォールバック
            if np.sum(use_mask) < 2:
                H_use = H
            else:
                H_use = H[use_mask]

    H_max = np.max(H_use)

    return H_max <= A * (1.0 + margin)



def compute_flux_sync_for_params(
    Omega_val: float,
    mu_s_hat_val: float,
    mu_k_hat_val: float,
    N_particles: int = 50,
    N_cycles: int = 50,
    x0_min: float = 0.0,
    x0_max: float = 2.0*np.pi,
    R_hat_val: float = R_hat,
    phi0_val: float = 0.0,
    energy_margin: float = 0.02,
    discard_fraction_for_mean: float = 0.0,  # ★追加：v_mean 計算時に捨てる比率
):
    """
    与えた (Ω, μ_s_hat, μ_k_hat) について、
    初期位相 x0 を [x0_min, x0_max) に一様に並べた N_particles 個の粒子を
    シミュレーションし、「エネルギー的に同期している粒子のみ」を使って
    フラックス

        Φ_sync = (1/N) Σ_i [ sync_i ? max(v_mean_i, 0) : 0 ]

    を計算して返す。

    戻り値:
        flux_sync : Φ_sync（同期粒子のみをカウントした右向きフラックス）
        v_means   : 各粒子の平均速度配列
        x0_list   : 各粒子の初期位相配列
        sync_flags: 各粒子が同期かどうか (True/False) の配列
    """

    # ---- グローバルパラメータの設定 ----
    globals()["Omega"]    = Omega_val
    globals()["mu_s_hat"] = mu_s_hat_val
    globals()["mu_k_hat"] = mu_k_hat_val

    # ---- 時間設定 ----
    t_end_hat = t_end_hat_from_cycles(N_cycles, Omega_val)
    dt_far_hat, dt_mid_hat, dt_near_hat, save_dt_hat = choose_dt_hat_for_Omega(Omega_val)

    # ---- 初期位相の一様サンプリング ----
    x0_list = np.linspace(x0_min, x0_max, N_particles, endpoint=False)

    v_means    = []
    sync_flags = []

    for idx, x0 in enumerate(x0_list):
        # 粒子を初期化
        p = Particle(
            x0=x0,
            z0=R_hat_val,
            vx0=0.0,
            vz0=0.0,
            R=R_hat_val,
            label=f"x0={x0:.3f}",
            color=None,
        )

        # シミュレーション
        hist_nd, events = simulate(
            p,
            t_end_hat=t_end_hat,
            dt_far_hat=dt_far_hat,
            dt_mid_hat=dt_mid_hat,
            dt_near_hat=dt_near_hat,
            save_dt_hat=save_dt_hat,
        )

        # 平均速度（最初の discard_fraction_for_mean 部分を捨てる）
        v_mean = compute_mean_velocity_from_hist(
            hist_nd,
            Lx=2.0*np.pi,
            discard_fraction=discard_fraction_for_mean,
        )
        v_means.append(v_mean)

        # エネルギー的な同期判定
        is_sync = is_synchronous_by_energy(
            hist_nd,
            Omega_val=Omega_val,
            R_hat_val=R_hat_val,
            phi0=phi0_val,
            margin=energy_margin,
            discard_fraction=discard_fraction_for_mean,
        )
        sync_flags.append(is_sync)

    v_means    = np.array(v_means,    dtype=float)
    sync_flags = np.array(sync_flags, dtype=bool)

    # 右向き同期粒子（フラックスに実際に寄与する粒子）
    mask_sync = sync_flags & (v_means > 0.0)

    # S_trap: 右向き同期粒子の割合
    if len(v_means) > 0:
        S_trap = np.mean(mask_sync)
    else:
        S_trap = 0.0

    # 同期粒子の平均速度 <v>_sync
    if np.any(mask_sync):
        v_sync = np.mean(v_means[mask_sync])
    else:
        v_sync = 0.0

    # フラックス (1/N Σ sync_i ? v_mean_i : 0) ・・・これまで通り
    v_contrib = np.where(mask_sync, v_means, 0.0)
    flux_vmean = np.mean(v_contrib)

    # Ω から計算した理論型フラックス S_trap * Ω も保存
    flux_omega = S_trap * Omega_val

    return flux_vmean, flux_omega, v_means, x0_list, sync_flags, S_trap, v_sync





Ωスイープ関数

In [ ]:
# ============================================================
#  摩擦係数を固定して Ω を sweep し、各種量を保存する関数
# ============================================================

def sweep_Omega_simulation(
    Omega_min: float = 0.5,
    Omega_max: float = 2.0,
    n_Omega: int = 15,
    mu_s: float = 0.0,
    mu_k: float = 0.0,
    N_particles: int = 50,
    N_cycles: int = 5,
    R_hat_val: float = R_hat,
    energy_margin: float = 0.0,
    discard_fraction_for_mean: float = 0.0,
    phi0_val: float = 0.0,
    stop_if_zero_flux: bool = False,   # 必要なら true にして早期終了
    flux_eps: float = 1e-3,
):
    """
    摩擦係数 (mu_s, mu_k) を固定して Ω をスイープし、
    各 Ω ごとに

        - flux_vmean : v_mean から計算したフラックス
        - flux_omega : S_trap * Ω で計算したフラックス
        - S_trap     : 右向き同期粒子の割合
        - v_sync     : 同期粒子の平均速度

    を評価する。

    戻り値:
        Omega_list       : np.array, 形状 (n_Omega,)
        flux_vmean_list  : np.array, 形状 (n_Omega,)
        flux_omega_list  : np.array, 形状 (n_Omega,)
        Strap_list       : np.array, 形状 (n_Omega,)
        v_sync_list      : np.array, 形状 (n_Omega,)
    """

    Omega_list       = np.linspace(Omega_min, Omega_max, n_Omega)
    flux_vmean_list  = []
    flux_omega_list  = []
    Strap_list       = []
    v_sync_list      = []

    #全Ωについての同期フラグを保存するリスト
    sync_flags_all = []
    x0_list_ref = None   # 最初のΩの x0_list を保存しておく

    print(f"=== Sweep Omega with fixed (mu_s, mu_k) = ({mu_s:.3f}, {mu_k:.3f}) ===")

    for i, Omega_val in enumerate(Omega_list, start=1):
        print(f"[{i}/{len(Omega_list)}]  Omega = {Omega_val:.4f}  "
              f"(mu_s={mu_s:.2f}, mu_k={mu_k:.2f}) ... ", end="")

        (flux_vmean,
         flux_omega,
         v_means,
         x0_list,
         sync_flags,
         S_trap,
         v_sync) = compute_flux_sync_for_params(
            Omega_val      = Omega_val,
            mu_s_hat_val   = mu_s,
            mu_k_hat_val   = mu_k,
            N_particles    = N_particles,
            N_cycles       = N_cycles,
            x0_min         = 0.0,
            x0_max         = 2.0*np.pi,
            R_hat_val      = R_hat_val,
            energy_margin  = energy_margin,
            phi0_val       = phi0_val,
            discard_fraction_for_mean = discard_fraction_for_mean,
        )

        flux_vmean_list.append(flux_vmean)
        flux_omega_list.append(flux_omega)
        Strap_list.append(S_trap)
        v_sync_list.append(v_sync)

        # 追加：同期フラグを保存
        sync_flags_all.append(sync_flags.astype(bool))
        if x0_list_ref is None:
            x0_list_ref = np.array(x0_list, dtype=float)

        print(f"flux_vmean = {flux_vmean:.6f},  S_trap = {S_trap:.3f},  v_sync = {v_sync:.3f}")

        # もし全区間ほぼ 0 になったら早期に打ち切るオプション
        if stop_if_zero_flux and abs(flux_vmean) < flux_eps:
            print(f"  → flux_vmean ≲ {flux_eps:.1e} なので、残りの Ω スイープを省略します。")
            # ここでループを抜けると、Omega_list の残りは使わない
            # 途中までの要素だけを返すように切り詰める
            Omega_list      = Omega_list[:i]
            break

    return (
        np.array(Omega_list,      dtype=float),
        np.array(flux_vmean_list, dtype=float),
        np.array(flux_omega_list, dtype=float),
        np.array(Strap_list,      dtype=float),
        np.array(v_sync_list,     dtype=float),
        x0_list_ref,                                  # ★ 追加
        np.array(sync_flags_all, dtype=bool),         # ★ 追加: shape (n_Omega, N_particles)
    )


μスイープ関数

In [ ]:
def sweep_mu_triangle_for_fixed_Omega(
    Omega_val: float,
    mu_s_start: float = 0.0,
    mu_s_end: float = 0.8,          # ← ここを「静止摩擦の理論限界」までにしておく
    dmu: float = 0.1,
    mu_k_max: float = 0.5,          # ★ 動摩擦係数 μk の最大値（0.5で打ち止め）
    N_particles: int = 50,
    N_cycles: int = 5,
    R_hat_val: float = R_hat,
    energy_margin: float = 0.0,
    discard_fraction_for_mean: float = 0.2,
    phi0_val: float = 0.0,
    stop_if_zero_flux: bool = True,  # フラックスが実質 0 になったら μs ループを打ち切る
    flux_eps: float = 1e-3,          # 「ゼロ」とみなすしきい値
):
    """
    Ω を固定して、μs を mu_s_start〜mu_s_end まで dmu 刻みで増やし、
    各 μs に対して μk = 0, dmu, ..., μs までスイープする。

    （μk の最大値を μs に制限した三角形領域を走査）
    """

    results = []

    # μs を 1 次元にスイープ
    mu_s_values = np.arange(mu_s_start, mu_s_end + 1e-9, dmu)

    print(f"=== Sweep (mu_s, mu_k) at fixed Omega={Omega_val:.3f} ===")

    x0_list_ref = None  # ★ 全ケースで共通の初期位相を 1 回だけ保存

    for i_s, mu_s_val in enumerate(mu_s_values, start=1):
        # この μs に対して μk は 0〜μs を dmu 刻み
        mu_k_upper  = min(mu_s_val, mu_k_max)
        mu_k_values = np.arange(0.0, mu_k_upper + 1e-9, dmu)

        flux_vmean_row = []
        flux_omega_row = []
        Strap_row      = []
        v_sync_row     = []

        sync_flags_row = []  # ★ この μs に対する「μkごとの同期フラグ」をためる
        print(f"\n[μs step {i_s}/{len(mu_s_values)}]  mu_s = {mu_s_val:.3f}")
        

        for i_k, mu_k_val in enumerate(mu_k_values, start=1):
            print(f"    (μk step {i_k}/{len(mu_k_values)})  mu_k = {mu_k_val:.3f} ... ", end="")

            (flux_vmean,
             flux_omega,
             v_means,
             x0_list,
             sync_flags,
             S_trap,
             v_sync) = compute_flux_sync_for_params(
                Omega_val      = Omega_val,
                mu_s_hat_val   = mu_s_val,
                mu_k_hat_val   = mu_k_val,
                N_particles    = N_particles,
                N_cycles       = N_cycles,
                x0_min         = 0.0,
                x0_max         = 2.0*np.pi,
                R_hat_val      = R_hat_val,
                energy_margin  = energy_margin,
                phi0_val       = phi0_val,
                discard_fraction_for_mean = discard_fraction_for_mean,
            )

            flux_vmean_row.append(flux_vmean)
            flux_omega_row.append(flux_omega)
            Strap_row.append(S_trap)
            v_sync_row.append(v_sync)

            # ★ 同期フラグを保存（各要素: shape (N_particles,)）
            sync_flags_row.append(np.array(sync_flags, dtype=bool))

            # ★ x0_list は全ケースで共通なので、最初の1回だけ保存
            if x0_list_ref is None:
                x0_list_ref = np.array(x0_list, dtype=float)

            print(f"flux_vmean = {flux_vmean:.6f},  S_trap = {S_trap:.3f},  v_sync = {v_sync:.3f}")

        flux_vmean_row = np.array(flux_vmean_row, dtype=float)
        flux_omega_row = np.array(flux_omega_row, dtype=float)
        Strap_row      = np.array(Strap_row,      dtype=float)
        v_sync_row     = np.array(v_sync_row,     dtype=float)

        # ★ sync_flags_row: list of (N_particles,) → array shape (n_mu_k, N_particles)
        if len(sync_flags_row) > 0:
            sync_flags_row = np.stack(sync_flags_row, axis=0)
        else:
            sync_flags_row = np.zeros((0, 0), dtype=bool)

        results.append({
            "mu_s": mu_s_val,
            "mu_k_list":       mu_k_values,
            "flux_vmean_list": flux_vmean_row,
            "flux_omega_list": flux_omega_row,
            "Strap_list":      Strap_row,
            "v_sync_list":     v_sync_row,
            "sync_flags_list": sync_flags_row,  # ★ 追加
            "x0_list":         x0_list_ref,     # ★ 追加
        })

        # ある μs 以上になると「全部ほぼ止まっている」なら μs スイープを打ち切る
        if stop_if_zero_flux and np.all(np.abs(flux_vmean_row) < flux_eps):
            print(f"\n→ 全ての μk で flux_vmean ≲ {flux_eps:.1e} となったので、"
                  f"mu_s = {mu_s_val:.3f} 以降のスイープを省略します。")
            break

    return results


理論からのフラックス計算関数

In [ ]:
# ===============================
# 1. パラメータ設定
# ===============================

# 例として、あなたの無次元半径 R_hat から A = exp(-R_hat) を定義
# （ここは状況に応じて好きに変えてOK）
A = np.exp(-R_hat)         # 有効ポテンシャル振幅 A

# Ω の掃引範囲：理論上 0 <= Ω <= 2*sqrt(A)
Omega_min = 0.0
Omega_max = 2.0 * np.sqrt(A)
n_Omega   = 5000            # 分解能（多めにしておくと綺麗なカーブになる）

# ===============================
# 2. フラックスの理論式（関数化）
# ===============================

def phi_sync(Omega, A):
    """
    同期粒子のみをカウントした理論フラックス Φ_sync(Ω) を返す。
    Ω, A はスカラー or NumPy 配列 OK。
    
    Φ_sync(Ω) = Ω * [ 1/2 + (1/π) * arcsin( 1 - Ω^2 / (2A) ) ]
    ただし、arcsin の引数が [-1,1] を外れたところでは Φ_sync = 0 とする。
    """
    Omega = np.asarray(Omega)
    
    # s_c = 1 - Ω^2 / (2A)
    s_c = 1.0 - Omega**2 / (2.0 * A)
    
    # 結果用配列を 0 で初期化
    Phi = np.zeros_like(Omega)
    
    # 有効な範囲（トラップあり）のマスク：-1 <= s_c <= 1
    mask = (s_c >= -1.0) & (s_c <= 1.0)
    
    # その範囲で S_trap と Φ_sync を計算
    S_trap = 0.5 + (1.0 / np.pi) * np.arcsin(s_c[mask])
    Phi[mask] = Omega[mask] * S_trap
    
    return Phi

# ===============================
# 3. グラフ用データ生成
# ===============================

Omega_vals = np.linspace(Omega_min, Omega_max, n_Omega)
Phi_vals   = phi_sync(Omega_vals, A)

# 最大値とその Ω も確認したければ
idx_max = np.argmax(Phi_vals)
Omega_max_flux = Omega_vals[idx_max]
Phi_max_flux   = Phi_vals[idx_max]
print(f"Omega_max (theory)  = {Omega_max_flux:.5f}")
print(f"Phi_sync_max (theory) = {Phi_max_flux:.5f}")

# ===============================
# 4. プロット
# ===============================

plt.figure(figsize=(6,4))
plt.plot(Omega_vals, Phi_vals)
plt.xlabel(r'$\Omega$', fontsize=12)
plt.ylabel(r'$\Phi_{\mathrm{sync}}(\Omega)$', fontsize=12)
plt.title(r'Theoretical flux of synchronous particles', fontsize=13)
plt.grid(True)
plt.tight_layout()
plt.show()


メイン

In [ ]:
# 例: 理論曲線から求めた「フラックス最大付近」の Ω
Omega0 = Omega_max_flux  # すでに理論計算で求めている値があるはず
# Omega0 = 1.7

# μs, μk スイープの設定
mu_s_start = 0.0
mu_s_end   = 0.5    # とりあえず 0.8 くらいまで。後で「理論 μs_crit」に置き換え。
dmu        = 0.1

N_particles = 100
N_cycles    = 20
discard_frac = 0.4

results_mu = sweep_mu_triangle_for_fixed_Omega(
    Omega_val  = Omega0,
    mu_s_start = mu_s_start,
    mu_s_end   = mu_s_end,
    dmu        = dmu,
    mu_k_max   = 0.5,       # ★動摩擦係数は 0.5 まで
    N_particles = N_particles,
    N_cycles    = N_cycles,
    R_hat_val   = R_hat,
    energy_margin = 0.005,
    discard_fraction_for_mean = discard_frac,
    phi0_val = phi0,
    stop_if_zero_flux = True,
    flux_eps = 1e-3,
)

In [ ]:
for entry in results_mu:
    print("mu_s =", entry["mu_s"])
    print("  mu_k_list        =", entry["mu_k_list"])
    print("  flux_vmean_list  =", entry["flux_vmean_list"])


In [ ]:
mu_s_block = results_mu[1]  # mu_s = 0.1 のデータ
x0_list    = mu_s_block["x0_list"]
mu_k_list  = mu_s_block["mu_k_list"]
sync_flags = mu_s_block["sync_flags_list"]  # shape (n_mu_k, N_particles)


In [ ]:
plt.figure(figsize=(8,5))

for i_k, mu_k in enumerate(mu_k_list):
    flags = sync_flags[i_k]  # (N_particles,) の True/False
    x0    = x0_list

    plt.scatter(
        x0[flags],           # 同期
        np.full(np.sum(flags), mu_k),
        color="green", s=20, label="sync" if i_k == 0 else ""
    )
    plt.scatter(
        x0[~flags],          # 非同期
        np.full(np.sum(~flags), mu_k),
        color="red", s=20, label="non-sync" if i_k == 0 else ""
    )

plt.xlabel("initial phase x0")
plt.ylabel("mu_k")
plt.yticks(mu_k_list)
plt.title(f"Sync map at mu_s = {mu_s_block['mu_s']}")
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
def plot_flux_vs_mu_k_for_each_mu_s(results_mu):
    plt.figure(figsize=(7, 5))

    for entry in results_mu:
        mu_s_val   = entry["mu_s"]
        mu_k_list  = entry["mu_k_list"]
        flux_list  = entry["flux_vmean_list"]

        plt.plot(mu_k_list, flux_list, marker="o", label=fr"$\mu_s={mu_s_val:.1f}$")

    plt.xlabel(r"$\mu_k$")
    plt.ylabel(r"Flux $\Phi_{\rm sync}$")
    plt.title(rf"Flux vs $\mu_k$ at fixed $\Omega={Omega0:.2f}$")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

# プロット
plot_flux_vs_mu_k_for_each_mu_s(results_mu)


In [ ]:
# μs=0.3 のフラックスを抽出してprint
mu_s_target = 0.2
for entry in results_mu:
    if abs(entry["mu_s"] - mu_s_target) < 1e-6:  # 浮動小数点誤差対策
        print(f"\n=== mu_s = {mu_s_target} のフラックス ===")
        print(f"mu_k values: {entry['mu_k_list']}")
        print(f"v_sync: {entry['v_sync_list']}")
        print(f"flux values: {entry['flux_list']}")
        for mu_k, flux in zip(entry['mu_k_list'], entry['flux_list']):
            print(f"  mu_k={mu_k:.1f}: flux_sync = {flux:.6f}")
        break


結果保存関数

In [ ]:
import pickle

# 例: このΩでの結果を保存
filename = f"results_mu_Omega_{Omega0:.3f}_mu_s0.1to0.4_new.pkl"
with open(filename, "wb") as f:
    pickle.dump(
        {
            "Omega0": Omega0,
            "results_mu": results_mu,
        },
        f
    )

print("saved to", filename)


In [ ]:
with open(filename, "rb") as f:
    data = pickle.load(f)

results_mu = data["results_mu"]

print(results_mu[0].keys())


ファイルくっつける

In [ ]:
import pickle

# --- 元ファイル名 ---
file1 = "results/results_mu_Omega_1.301_mu_s0.00to0.5.pkl"
file2 = "results/results_mu_Omega_1.301_mu_s0.6to1.0.pkl"

# --- 読み込み ---
with open(file1, "rb") as f:
    data1 = pickle.load(f)

with open(file2, "rb") as f:
    data2 = pickle.load(f)

results1 = data1["results_mu"]
results2 = data2["results_mu"]

Omega0_1 = data1["Omega0"]
Omega0_2 = data2["Omega0"]

# --- Ω が一致しているか一応確認 ---
if abs(Omega0_1 - Omega0_2) > 1e-12:
    print("Warning: Omega0 values differ!", Omega0_1, Omega0_2)

# --- μs の順に結合（ソート付き） ---
combined = results1 + results2

# μs が昇順になるように並べ替え
combined_sorted = sorted(combined, key=lambda d: d["mu_s"])

# --- 保存 ---
output = {
    "Omega0": Omega0_1,
    "results_mu": combined_sorted,
}

outfile = "results/results_mu_Omega_1.301_full.pkl"

with open(outfile, "wb") as f:
    pickle.dump(output, f)

print("Saved combined file:", outfile)


ファイル呼び出し関数

In [ ]:
import pickle

filename = "results/results_mu_Omega_1.700_mu_s0.1to0.4_new.pkl"  # 実際に保存したファイル名
with open(filename, "rb") as f:
    data = pickle.load(f)

Omega0_loaded  = data["Omega0"]
results_mu_old = data["results_mu"]

# さっき作ったプロット関数を再利用すればOK
# plot_flux_vs_mu_k_for_each_mu_s(results_mu_old, Omega0_loaded)


可視化

In [ ]:
plt.figure(figsize=(7,5))

for entry in results_mu_old:
    mu_s = entry["mu_s"]
    if mu_s >= 0.95:   # μs=1.0 をスキップ
        continue

    plt.plot(entry["mu_k_list"],
             entry["flux_omega_list"],
             marker="o",
             label=f"mu_s={mu_s:.1f}")

plt.xlabel(r"$\mu_k$")
plt.ylabel("Flux_vmean")
plt.title("Flux_vmean vs mu_k (for each mu_s)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(7,5))
for entry in results_mu_old:
    mu_s = entry["mu_s"]
    if mu_s >= 0.95:   # μs=1.0 をスキップ
        continue

    plt.plot(entry["mu_k_list"],
             entry["Strap_list"],
             marker="o",
             label=f"mu_s={mu_s:.1f}")

plt.xlabel(r"$\mu_k$")
plt.ylabel(r"$S_{\mathrm{trap}}$")
plt.title("S_trap vs mu_k for each mu_s")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# まず全点を1つの配列にまとめる
mu_s_all = []
mu_k_all = []
flux_all = []

for entry in results_mu_old:
    mu_s = entry["mu_s"]
    for mu_k, flux in zip(entry["mu_k_list"], entry["flux_vmean_list"]):
        mu_s_all.append(mu_s)
        mu_k_all.append(mu_k)
        flux_all.append(flux)

mu_s_all = np.array(mu_s_all)
mu_k_all = np.array(mu_k_all)
flux_all = np.array(flux_all)

plt.figure(figsize=(6,5))
sc = plt.scatter(mu_k_all, mu_s_all,
                 c=flux_all,
                 cmap="viridis",
                 s=120, marker="s", edgecolors="k", linewidths=0.3)

plt.colorbar(sc, label="Flux_vmean")
plt.xlabel(r"$\mu_k$")
plt.ylabel(r"$\mu_s$")
plt.title("Flux_vmean at each (mu_s, mu_k)")
plt.xticks(np.arange(0.0, 0.51, 0.1))
plt.yticks(sorted({entry["mu_s"] for entry in results_mu_old}))
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
mu_s_all = []
mu_k_all = []
strap_all = []

for entry in results_mu_old:
    mu_s = entry["mu_s"]
    for mu_k, s in zip(entry["mu_k_list"], entry["Strap_list"]):
        mu_s_all.append(mu_s)
        mu_k_all.append(mu_k)
        strap_all.append(s)

mu_s_all = np.array(mu_s_all)
mu_k_all = np.array(mu_k_all)
strap_all = np.array(strap_all)

plt.figure(figsize=(6,5))
sc = plt.scatter(mu_k_all, mu_s_all,
                 c=strap_all,
                 cmap="viridis",
                 s=120, marker="s", edgecolors="k", linewidths=0.3)

plt.colorbar(sc, label=r"$S_{\mathrm{trap}}$")
plt.xlabel(r"$\mu_k$")
plt.ylabel(r"$\mu_s$")
plt.title(r"$S_{\mathrm{trap}}$ at each (mu_s, mu_k)")
plt.xticks(np.arange(0.0, 0.51, 0.1))
plt.yticks(sorted({entry["mu_s"] for entry in results_mu_old}))
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd

# 行方向: mu_s
mu_s_vals = np.array([entry["mu_s"] for entry in results_mu_old])

# 列方向: mu_k は 0,0.1,...,0.5 でそろっている前提
dmu = 0.1
mu_k_vals = np.arange(0.0, 0.5 + 1e-9, dmu)

# NaN で初期化
flux_grid  = np.full((len(mu_s_vals), len(mu_k_vals)), np.nan)
strap_grid = np.full((len(mu_s_vals), len(mu_k_vals)), np.nan)

# 埋める
for i, entry in enumerate(results_mu_old):
    mu_s = entry["mu_s"]
    for mu_k, flux, strap in zip(entry["mu_k_list"],
                                 entry["flux_vmean_list"],
                                 entry["Strap_list"]):
        j = int(round(mu_k / dmu))   # mu_k=0.0→0列, 0.1→1列,...
        flux_grid[i, j]  = flux
        strap_grid[i, j] = strap

# DataFrame にして表示（小数3桁で丸め）
df_flux  = pd.DataFrame(flux_grid,  index=mu_s_vals, columns=mu_k_vals)
df_strap = pd.DataFrame(strap_grid, index=mu_s_vals, columns=mu_k_vals)

print("=== Flux_vmean table (rows: mu_s, cols: mu_k) ===")
print(df_flux.round(3))

print("\n=== S_trap table (rows: mu_s, cols: mu_k) ===")
print(df_strap.round(3))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 呼び出したファイルから:横軸=静止摩擦係数(mu_s), 縦軸=S_trap, 色=動摩擦係数(mu_k)

# results_mu_old から全データを抽出
mu_s_list = []
strap_list = []
mu_k_list = []

for entry in results_mu_old:
    mu_s = entry["mu_s"]
    for mu_k, strap in zip(entry["mu_k_list"], entry["Strap_list"]):
        mu_s_list.append(mu_s)
        strap_list.append(strap)
        mu_k_list.append(mu_k)

mu_s_arr = np.array(mu_s_list)
strap_arr = np.array(strap_list)
mu_k_arr = np.array(mu_k_list)

# 動摩擦係数の一意な値を取得
mu_k_unique = np.unique(mu_k_arr)

# プロット
fig, ax = plt.subplots(figsize=(10, 6))

# カラーマップを設定
cmap = plt.cm.tab20  # または viridis, coolwarm など
colors = cmap(np.linspace(0, 1, len(mu_k_unique)))

# 各 mu_k 値ごとにプロット
for idx, mu_k_val in enumerate(mu_k_unique):
    mask = mu_k_arr == mu_k_val
    ax.plot(mu_s_arr[mask], strap_arr[mask],
            marker='o', markersize=8, linewidth=2,
            label=f'$\mu_k = {mu_k_val:.1f}$',
            color=colors[idx])

ax.set_xlabel(r'$\mu_s$ (Static friction coefficient)', fontsize=12)
ax.set_ylabel(r'$S_{\mathrm{trap}}$ (Trapped particle fraction)', fontsize=12)
ax.set_title(f'$S_{{\\mathrm{{trap}}}}$ vs $\mu_s$ at $\Omega = {Omega0:.3f}$', fontsize=13)
ax.grid(True, alpha=0.3)
ax.legend(loc='best', fontsize=10)
ax.set_ylim(bottom=0)

plt.tight_layout()
plt.show()


同期の可視化

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def get_sync_data_for_mu(results_mu, mu_s_target, mu_k_target, tol=1e-8):
    """
    results_mu（sweep_mu_triangle_for_fixed_Omega の結果）から
    指定した (μs, μk) に対応する
      x0_list : 初期位相 (0〜2π)
      sync_flags : True=同期, False=非同期
    を取り出す。
    """
    # μs で該当エントリを探す
    for entry in results_mu:
        if abs(entry["mu_s"] - mu_s_target) < tol:
            mu_k_list = np.asarray(entry["mu_k_list"])
            # μk のインデックス
            idx_k = np.where(np.abs(mu_k_list - mu_k_target) < tol)[0]
            if len(idx_k) == 0:
                raise ValueError(f"mu_k={mu_k_target} が mu_s={mu_s_target} の中にありません")
            idx_k = idx_k[0]

            # 同期フラグと x0_list が保存されている前提
            if "sync_flags_list" not in entry or entry["sync_flags_list"] is None:
                raise ValueError("この results_mu には 'sync_flags_list' が入っていません。"
                                 "mu スイープを sync_flags 保存版の関数で再計算する必要があります。")

            sync_flags_list = np.asarray(entry["sync_flags_list"])   # shape (n_mu_k, N_particles)
            x0_list = np.asarray(entry["x0_list"])                   # shape (N_particles,)

            sync_flags = sync_flags_list[idx_k]                      # この μk の列だけ抜き出す
            return x0_list, sync_flags

    raise ValueError(f"mu_s={mu_s_target} のエントリが見つかりません")

import numpy as np
import matplotlib.pyplot as plt



In [ ]:
def plot_sync_band_for_mu_s(results_mu, mu_s_target, tol=1e-8):
    SYNC_COLOR = "tab:blue"
    NON_COLOR  = "gray"

    # -------------------------
    # μs に対応するエントリ探索
    # -------------------------
    entry = None
    for e in results_mu:
        if abs(e["mu_s"] - mu_s_target) < tol:
            entry = e
            break
    if entry is None:
        raise ValueError(f"mu_s={mu_s_target} が見つかりません")

    mu_k_list       = np.asarray(entry["mu_k_list"])
    x0_list         = np.asarray(entry["x0_list"])
    sync_flags_list = np.asarray(entry["sync_flags_list"])

    x0_mod = (x0_list + 2*np.pi) % (2*np.pi)

    plt.figure(figsize=(10,4))

    for j, mu_k in enumerate(mu_k_list):
        sync_flags = sync_flags_list[j]
        y = np.full_like(x0_mod, mu_k, dtype=float)

        # sync → 青
        plt.scatter(x0_mod[sync_flags], y[sync_flags],
                    marker="s", s=20,
                    color=SYNC_COLOR,
                    label="sync" if j==0 else None)

        # non-sync → 灰枠のみ
        plt.scatter(x0_mod[~sync_flags], y[~sync_flags],
                    marker="s", s=20,
                    facecolors="none", edgecolors=NON_COLOR,
                    label="non-sync" if j==0 else None)

    xticks = [0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi]
    xtick_labels = ["0", r"$\pi/2$", r"$\pi$", r"$3\pi/2$", r"$2\pi$"]
    plt.xticks(xticks, xtick_labels)

    plt.xlabel(r"initial phase $x_0$")
    plt.ylabel(r"kinetic friction $\mu_k$")
    plt.title(rf"Sync vs $x_0$  (μ_s={mu_s_target:.2f})")
    plt.grid(True, axis="y", linestyle="--", alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
plot_sync_band_for_mu_s(results_mu_old, mu_s_target=0.2)


In [ ]:
def plot_sync_on_potential(results_mu, mu_s_target, mu_k_target,
                           A, phi0=0.0, tol=1e-8):
    """
    指定した (μs, μk) に対して、
      有効ポテンシャル V(ψ) = A sin ψ 上に
      同期/非同期の初期点 (ψ0, V(ψ0)) を重ねて描く。
    ψ0 = x0 + φ0 を [-π, π] に折り返して使う。
    """
    # すでに定義している get_sync_data_for_mu を利用
    x0_list, sync_flags = get_sync_data_for_mu(results_mu, mu_s_target, mu_k_target, tol=tol)

    x0 = np.asarray(x0_list)

    # ψ0 = x0 + φ0 を [-π, π] に折り返し
    psi0 = (x0 + phi0 + np.pi) % (2*np.pi) - np.pi

    # ポテンシャル V(ψ) の曲線
    psi = np.linspace(-np.pi, np.pi, 400)
    V   = A * np.sin(psi)

    V0  = A * np.sin(psi0)

    plt.figure(figsize=(6,4))
    plt.plot(psi, V, label=r"$V(\psi)=A\sin\psi$", linewidth=2)

    # 同期
    plt.scatter(psi0[sync_flags], V0[sync_flags],
                marker="o", s=35, label="sync")

    # 非同期
    plt.scatter(psi0[~sync_flags], V0[~sync_flags],
                marker="x", s=40, label="non-sync")

    xticks = [-np.pi, -0.5*np.pi, 0, 0.5*np.pi, np.pi]
    xtick_labels = [r"$-\pi$", r"$-\pi/2$", r"0", r"$\pi/2$", r"$\pi$"]
    plt.xticks(xticks, xtick_labels)

    plt.xlabel(r"$\psi$")
    plt.ylabel(r"$V(\psi)$")
    plt.title(rf"Initial points on potential ($\mu_s={mu_s_target:.2f}, \mu_k={mu_k_target:.2f}$)")
    plt.grid(True, alpha=0.4)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
A = np.exp(-R_hat)

# 例: μs=0.3, μk=0.2 のポテンシャル上の分布
plot_sync_on_potential(results_mu_old,
                       mu_s_target=0.2,
                       mu_k_target=0.1,
                       A=A,
                       phi0=phi0)


In [ ]:
def summarize_sync_regions(results_mu, mu_s_target, mu_k_target, tol=1e-8):
    x0_list, sync_flags = get_sync_data_for_mu(results_mu, mu_s_target, mu_k_target, tol=tol)

    # x0 をソート（念のため）
    order = np.argsort(x0_list)
    x0 = x0_list[order]
    flags = sync_flags[order]

    def collect_regions(mask):
        regions = []
        start = None
        prev_idx = None

        for i, (xi, m) in enumerate(zip(x0, mask)):
            if m:
                if start is None:
                    start = xi
                    prev_idx = i
                else:
                    if i != prev_idx + 1:  # ひと区切り
                        regions.append((start, x0[prev_idx]))
                        start = xi
                    prev_idx = i
            else:
                if start is not None:
                    regions.append((start, x0[prev_idx]))
                    start = None
                    prev_idx = None

        if start is not None:
            regions.append((start, x0[prev_idx]))

        return regions

    sync_regions  = collect_regions(flags)
    async_regions = collect_regions(~flags)

    print(f"=== sync regions for mu_s={mu_s_target:.2f}, mu_k={mu_k_target:.2f} ===")
    for a, b in sync_regions:
        print(f"  x0 in [{a:.3f}, {b:.3f}] rad")

    print(f"\n=== non-sync regions for mu_s={mu_s_target:.2f}, mu_k={mu_k_target:.2f} ===")
    for a, b in async_regions:
        print(f"  x0 in [{a:.3f}, {b:.3f}] rad")


In [ ]:
summarize_sync_regions(results_mu_old, mu_s_target=0.2, mu_k_target=0.1)
